In [1]:
# 读入Fluit数据集 import
import os 
import os.path 
import numpy as np

base_path="../../../dataset/FluidDataset/"

In [2]:
# 导入基础的三种台阶高,及对应路径
Paths={}
heights=np.array(range(1,4),dtype=np.float32)*0.05
for i in range(0,3):
    Paths[str(heights[i])]=base_path+str(heights[i])+"/"
Paths


{'0.05': '../../../dataset/FluidDataset/0.05/',
 '0.1': '../../../dataset/FluidDataset/0.1/',
 '0.15': '../../../dataset/FluidDataset/0.15/'}

In [3]:
# 对特定路径导入全部数据到数组
Datas={}


def DataLoaderFromFiles(path="../../../dataset/FluidDataset/0.05/", height=0.05, set_type="training", vars_type="input", speed_add_flag=False, SpeedList=[]):
    filepath=path+set_type+"_set/"+vars_type+"/"
    filelists=os.listdir(filepath)
    
    for filename in filelists:
        txt_data=np.loadtxt(filepath+filename)
        if speed_add_flag:
            SpeedList.append(float(filename[:-4]))
        Datas[str(height)+set_type+vars_type+filename[:-4]]=(height,vars_type,float(filename[:-4]),txt_data)
            

In [4]:
# 导入全部数据
vars_types=["input","input2","input3","k","p","u","uv","v","epsilon"]
set_types=["training","development","test"]
SpeedListDict={}
for height in heights:
    for set_type in set_types:
        SpeedList=[]
        for vars_type in vars_types:
            speed_add_flag=False
            # 只进行一次速度加列表
            if vars_type == vars_types[0]:
                speed_add_flag=True
            DataLoaderFromFiles(Paths[str(height)],height,set_type,vars_type,speed_add_flag,SpeedList)
        SpeedListDict[str(height)+set_type]=SpeedList


In [6]:
SpeedListDict

{'0.05training': [0.02,
  0.04,
  0.05,
  0.07,
  0.09,
  0.1,
  0.12,
  0.14,
  0.15,
  0.17,
  0.19,
  0.2,
  0.22,
  0.24,
  0.25,
  0.27,
  0.29,
  0.3,
  0.32,
  0.34,
  0.35,
  0.37,
  0.39,
  0.4,
  0.42,
  0.44,
  0.45,
  0.47,
  0.49,
  0.5,
  0.52,
  0.54,
  0.55,
  0.57,
  0.59,
  0.6,
  0.62,
  0.64,
  0.65,
  0.67,
  0.69,
  0.7,
  0.72,
  0.74,
  0.75,
  0.77,
  0.79,
  0.8,
  0.82,
  0.84,
  0.85,
  0.87,
  0.89,
  0.9,
  0.92,
  0.94,
  0.95,
  0.97,
  0.99,
  1.02,
  1.04,
  1.05,
  1.07,
  1.09,
  1.1,
  1.12,
  1.14,
  1.15,
  1.17,
  1.19,
  1.2,
  1.22,
  1.24,
  1.25,
  1.27,
  1.29,
  1.3,
  1.32,
  1.34,
  1.35,
  1.37,
  1.39,
  1.4,
  1.42,
  1.44,
  1.45,
  1.47,
  1.49,
  1.5,
  1.52,
  1.54,
  1.55,
  1.57,
  1.59,
  1.6,
  1.62,
  1.64,
  1.65,
  1.67,
  1.69,
  1.7,
  1.72,
  1.74,
  1.75,
  1.77,
  1.79,
  1.8,
  1.82,
  1.84,
  1.85,
  1.87,
  1.89,
  1.9,
  1.92,
  1.94,
  1.95,
  1.97,
  1.99,
  1.0,
  10.02,
  10.04,
  10.05,
  10.07,
  10.09,
  10.1

In [7]:
Datas

{'0.05traininginput0.02': (0.05,
  'input',
  0.02,
  array([[0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
         [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
         [0.02, 0.02, 0.02, ..., 0.02, 0.02, 0.02],
         ...,
         [0.  , 0.  , 0.  , ..., 0.02, 0.02, 0.02],
         [0.  , 0.  , 0.  , ..., 0.02, 0.02, 0.02],
         [0.  , 0.  , 0.  , ..., 0.02, 0.02, 0.02]])),
 '0.05traininginput0.04': (0.05,
  'input',
  0.04,
  array([[0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
         [0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
         [0.04, 0.04, 0.04, ..., 0.04, 0.04, 0.04],
         ...,
         [0.  , 0.  , 0.  , ..., 0.04, 0.04, 0.04],
         [0.  , 0.  , 0.  , ..., 0.04, 0.04, 0.04],
         [0.  , 0.  , 0.  , ..., 0.04, 0.04, 0.04]])),
 '0.05traininginput0.05': (0.05,
  'input',
  0.05,
  array([[0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
         [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
         [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
         ...,
         [0.  , 